# CityBikes

In [2]:
import pandas as pd
import requests

# Base endpoint = http://api.citybik.es/v2/networks

url = 'http://api.citybik.es/v2/networks'

Send a request to CityBikes for the city of your choice. 

In [3]:
# An example url is given - http://api.citybik.es/v2/networks?fields=id,name,href
# We can get an idea of how the parameters are structured.

params = {
    'fields':'id,name,href'
}

response = requests.get(url= url, params=params)
response.json()
# This output tells us each city/point has its own network(s) identified by <bike sharing company>-<city name>
# The href is the base endpoint + the id. I want to use Berlin and the map in the API documentation has 'Nextbike' associated with Berlin.
# The network for Berlin should then be '.../v2/networks/nextbike-berlin'
# CTRL+F in the output confirms this.

{'networks': [{'href': '/v2/networks/velobike-moscow',
   'id': 'velobike-moscow',
   'name': 'Velobike'},
  {'href': '/v2/networks/baerum-bysykkel',
   'id': 'baerum-bysykkel',
   'name': 'Bysykkel'},
  {'href': '/v2/networks/bicincitta-siena',
   'id': 'bicincitta-siena',
   'name': 'Bicincittà'},
  {'href': '/v2/networks/cyclopolis-maroussi',
   'id': 'cyclopolis-maroussi',
   'name': 'Cyclopolis'},
  {'href': '/v2/networks/cycle-hire-slough',
   'id': 'cycle-hire-slough',
   'name': 'Cycle Hire'},
  {'href': '/v2/networks/cyclopolis-nafplio',
   'id': 'cyclopolis-nafplio',
   'name': 'Cyclopolis'},
  {'href': '/v2/networks/bicincitta-parco-dei-colli-di-bergamo',
   'id': 'bicincitta-parco-dei-colli-di-bergamo',
   'name': 'Bicincittà'},
  {'href': '/v2/networks/cyclopolis-aigialeia',
   'id': 'cyclopolis-aigialeia',
   'name': 'Cyclopolis'},
  {'href': '/v2/networks/cyclopolis-marathon',
   'id': 'cyclopolis-marathon',
   'name': 'Cyclopolis'},
  {'href': '/v2/networks/bicincitta-m

In [4]:
berlin = '/nextbike-berlin'
# Test a response to Berlin's network
response = requests.get(url= url + berlin)
response.json()
# It works and has spit out all the relevant stations.

{'network': {'company': ['Nextbike GmbH'],
  'href': '/v2/networks/nextbike-berlin',
  'id': 'nextbike-berlin',
  'location': {'city': 'Berlin',
   'country': 'DE',
   'latitude': 52.5087,
   'longitude': 13.3563},
  'name': 'Nextbike',
  'stations': [{'empty_slots': 0,
    'extra': {'bike_uids': ['18114', '18107', '15704', '19744'],
     'number': '1640',
     'slots': 4,
     'uid': '44437'},
    'free_bikes': 4,
    'id': 'f5462555877f4cb1317baf6041527a93',
    'latitude': 52.504157,
    'longitude': 13.335328,
    'name': 'virtuell - Kurfürstendamm/Rankestraße',
    'timestamp': '2023-01-27T09:44:17.406000Z'},
   {'empty_slots': 4,
    'extra': {'bike_uids': [''], 'number': '1670', 'slots': 4, 'uid': '72228'},
    'free_bikes': 0,
    'id': 'c32093893e4e32afee18894c7c0c1ba4',
    'latitude': 52.496986,
    'longitude': 13.29121,
    'name': 'virtuell - EDEKA Schmitt (S Halensee)',
    'timestamp': '2023-01-27T09:44:17.415000Z'},
   {'empty_slots': 2,
    'extra': {'bike_uids': ['17

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [5]:
# return lat, long, slots, and I'm going to grab free_bikes and empty_slots as well as that seems important.
url = 'http://api.citybik.es/v2/networks'
berlin = '/nextbike-berlin'

response = requests.get(url= (url + berlin))

bike_json = response.json() # Cache our response so we can play around with it and get an understanding of how the Json is laid out.


In [6]:
print('Lat -',bike_json['network']['stations'][0]['latitude'])
print('Long -',bike_json['network']['stations'][0]['longitude'])
print('Slots -',bike_json['network']['stations'][0]['extra']['slots'])
print('Bikes available -',bike_json['network']['stations'][0]['free_bikes'])
print('Bikes in use -',bike_json['network']['stations'][0]['empty_slots'])

# Create for loop to iterate over station list in bike_json (where the [0] is in the above indexing)
# This will grab this info for all stations in Berlin.


Lat - 52.504157
Long - 13.335328
Slots - 4
Bikes available - 4
Bikes in use - 0


Put your parsed results into a DataFrame.

In [7]:
# relevant info from each station will be concatenated into 'berlin_compiled' before it is furthered processed.
berlin_compiled= pd.DataFrame()

for station in bike_json['network']['stations']:
    tmp_df = pd.DataFrame()
    # This df will be constructed into the df that will be stacked onto 'berlin_compiled'
    tmp_df = pd.concat([
        tmp_df,
        pd.Series(station['latitude'],name='lat'),
        pd.Series(station['longitude'],name='long'),
        pd.Series(station['free_bikes'],name='bikes_available'),
        pd.Series(station['empty_slots'],name='bikes_in_use'),
        pd.Series(station['extra']['slots'],name='total_bikes')
    ],
    axis=1 # horizontal concat
    )
    berlin_compiled = pd.concat([berlin_compiled,tmp_df]) # vertical concat
    
    # essentially we're creating 'berlin_compiled' one row at a time.


In [8]:
berlin_bikes = berlin_compiled.reset_index(drop=True) # set a proper incrementing index instead of all 0s
berlin_bikes
# Now we have a df of relevant information for all the stations in Berlin.

,lat,long,bikes_available,bikes_in_use,total_bikes
0,52.504157,13.335328,4,0,4
1,52.496986,13.291210,0,4,4
2,52.498323,13.296157,3,2,5
3,52.499137,13.303743,4,1,5
4,52.500614,13.294401,6,2,8
...,...,...,...,...,...
1803,52.465047,13.436931,0,4,4
1804,52.511428,13.445441,1,3,4
1805,52.509810,13.377490,0,4,4
1806,52.516330,13.381310,0,4,4


In [9]:
berlin_bikes.to_csv('../data/berlin_bikes.csv')
# Export table for Part 2 of this project.